In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns
import pickle
import time
# from tqdm import tqdm

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [2]:
train = pd.read_csv('tr_preprocessing_model3_new.csv')

train.shape
train.head()

(2724068, 155)

,instance_id,item_id,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,user_gender_id,user_age_level,user_occupation_id,user_star_level,context_id,context_timestamp,context_page_id,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,is_trade,time,day,hour,minute,user_query_day,user_query_day_hour,user_query_minute,item_category_split_count,item_property_split_count,pred_category_split_count,pred_property_split_count,item_pred_category_score,item_pred_property_score,item_pred_category_score_item%,item_pred_property_score_item%,item_pred_category_score_pred%,item_pred_property_score_pred%,user_shop_count,user_item_count,user_context_count,user_shop_trade_count,user_item_trade_count,user_context_trade_count,user_brand_count,user_city_count,user_brand_trade_count,user_city_trade_count,item_occupation_count,item_age_count,item_gender_count,item_occupation_trade_count,item_age_trade_count,item_gender_trade_count,shop_score,item_score,item_score2,item_score3,item_hot,user_hot,shop_hot,brand_hot,occupation_hot,city_hot,item_trade_hot,user_trade_hot,shop_trade_hot,brand_trade_hot,occupation_trade_hot,city_trade_hot,user_sell_power,user_sell_power_mean,user_sell_power_mean_item_minus,user_sell_trade_power,user_sell_trade_power_mean,user_sell_trade_power_mean_item_minus,yesterday,yesterhour,user_day_query,user_yesterday_query,user_hour_query,user_yesterhour_query,user_minute_query,item_day_query,item_hour_query,item_minute_query,user_item_day_query,user_item_hour_query,user_item_minute_query,min_time,user_is_his,item_is_his,brand_is_his,user_is_trade_his,item_is_trade_his,brand_is_trade_his,item_trade_percent,brand_trade_percent,user_trade_percent,occupation_trade_percent,is_home,shop_item_count,brand_item_count,city_brand_count,city_item_count,shop_item_count_his,brand_item_count_his,city_brand_count_his,city_item_count_his,user_count,user_day_count,user_hour_count,user_minute_count,user_next_count,user_last_count,user_next_time,user_last_time,next_time_sub,last_time_sub,user_item_num,user_shop_num,user_item_day_num,user_shop_day_num,user_item_num_next,user_item_num_last,user_shop_num_next,user_shop_num_last,user_item_next_time,user_item_last_time,user_shop_next_time,user_shop_last_time,user_item_next_time_sub,user_item_last_time_sub,user_shop_next_time_sub,user_shop_last_time_sub,user_fisrt_num,user_item_fisrt_num,user_shop_fisrt_num,user_count_first_time,user_item_count_first_time,user_shop_count_first_time,user_count_first_time_sub,user_item_count_first_time_sub,user_shop_count_first_time_sub,max_click,is_last_click,max_user_item_click,is_last_user_item_click,max_click_time,max_user_item_click_time,max_click_time_sub,max_user_item_click_time_sub
0,1448993413723047248,7402285735527989242,2080427454881213041,3122721854741763495,7,8,8,15,3347527000263617499,1,1005,2002,3005,2887158526846708205,1536163200,12,4350321630089728797,8,0.996139,7,0.990189,0.991698,0.993208,0,2018-09-06 00:00:00,6,0,5,4,2,2,2,22,3,2,2,1,6.666667,0.434783,5.000000,3.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,21.0,29.0,0.0,1.0,1.0,14.599349,128,14.222222,1.015873,107.0,4.0,120.0,120.0,2484989,133820.0,3.0,0.0,3.0,3.0,29558,1636.0,30.0,6.000000,1.000000,0.0,0.0,7.0,5,23,4,0.0,2,0.0,2,42,8,3,1,1,1,0.0,0,0,1,1,0,0,2.803738,2.500000,0.0,1.189462,0,2,2,168,784,3.0,3.0,190.0,1115.0,0,0,0,0,1,-1,2018-09-06 00:01:57,NaN,117,999999,0,0,0,0,1,-1,1,-1,NaN,NaN,NaN,NaN,999999,999999,999999,999999,0,0,0,2018-09-06 00:00:00,2018-09-06 00:00:00,2018-09-06 00:00:00,0,0,0,3,0,0,1,2018-09-06 19:25:12,2018-09-06 00:00:00,69912,0
1,5165341565614770629,295933528251432883,562716190599450643,5918626470536001929,8,6,12,15,5221645665268099566,0,1004,2005,3005,7744811423075127265,1536163200,0,1052887836978786251,15,1.000000,14,0.977175,0.979435,0.980210,0,2018-09-06 00:00:00,6,0,5,2,1,1,2,25,2,2,2,1,6.666667,0.384615,6.666667,3.333333,0.0,0.0,0.0,

In [3]:
origin = [
    'item_id', 'item_brand_id', 'item_city_id',
    'item_price_level', 'item_sales_level', 'item_collected_level',
    'item_pv_level', 'user_id', 'user_gender_id', 'user_age_level',
    'user_occupation_id', 'user_star_level', 'context_id', 'context_timestamp',
    'context_page_id', 'shop_id', 'shop_review_num_level',
    'shop_review_positive_rate', 'shop_star_level', 'shop_score_service',
    'shop_score_delivery', 'shop_score_description'
]

predictors = [
    'day', 'hour', 'minute', 'user_query_day', 'user_query_day_hour',
    'user_query_minute', 'item_category_split_count',
    'item_property_split_count', 'pred_category_split_count',
    'pred_property_split_count', 'item_pred_category_score',
    'item_pred_property_score', 'item_pred_category_score_item%',
    'item_pred_property_score_item%', 'item_pred_category_score_pred%',
    'item_pred_property_score_pred%', 'user_shop_count', 'user_item_count',
    'user_context_count', 'user_shop_trade_count', 'user_item_trade_count',
    'user_context_trade_count', 'user_brand_count', 'user_city_count',
    'user_brand_trade_count', 'user_city_trade_count', 'item_occupation_count',
    'item_age_count', 'item_gender_count', 'item_occupation_trade_count',
    'item_age_trade_count', 'item_gender_trade_count', 'shop_score',
    'item_score', 'item_score2', 'item_score3', 'item_hot', 'user_hot',
    'shop_hot', 'brand_hot', 'occupation_hot', 'city_hot', 'item_trade_hot',
    'user_trade_hot', 'shop_trade_hot', 'brand_trade_hot',
    'occupation_trade_hot', 'city_trade_hot', 'user_sell_power',
    'user_sell_power_mean', 'user_sell_power_mean_item_minus',
    'user_sell_trade_power', 'user_sell_trade_power_mean',
    'user_sell_trade_power_mean_item_minus', 'yesterday', 'yesterhour',
    'user_day_query', 'user_yesterday_query', 'user_hour_query',
    'user_yesterhour_query', 'user_minute_query', 'item_day_query',
    'item_hour_query', 'item_minute_query', 'user_item_day_query',
    'user_item_hour_query', 'user_item_minute_query', 'min_time',
    'user_is_his', 'item_is_his', 'brand_is_his', 'user_is_trade_his',
    'item_is_trade_his', 'brand_is_trade_his', 'user_count', 'user_day_count',
    'user_hour_count', 'user_minute_count', 'item_trade_percent',
    'brand_trade_percent', 'user_trade_percent', 'occupation_trade_percent',
    'is_home', 'shop_item_count', 'brand_item_count', 'city_brand_count',
    'city_item_count', 'shop_item_count_his', 'brand_item_count_his',
    'city_brand_count_his', 'city_item_count_his', 'next_time_sub',
    'last_time_sub', 'user_item_num', 'user_shop_num', 'user_item_day_num',
    'user_shop_day_num', 'user_item_next_time_sub', 'user_item_last_time_sub',
    'user_shop_next_time_sub', 'user_shop_last_time_sub',
    'user_count_first_time_sub', 'user_item_count_first_time_sub',
    'user_shop_count_first_time_sub', 'max_click', 'is_last_click',
    'max_user_item_click', 'is_last_user_item_click', 'max_click_time_sub',
    'max_user_item_click_time_sub'
]

by = ['instance_id']

target = ['is_trade']

In [4]:
import sys
sys.path.append('/Users/zero/xgboost/python-package')
import xgboost as xgb
# import lightgbm as lgb

from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import KFold
from sklearn.metrics import log_loss

/home/user/cuser/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
test = train[train.is_trade == -1]
val = train[(train.is_trade != -1) & (train.day == 7) & (train.hour >= 10) & (train.hour <= 12)]
train = train[(train.is_trade != -1) & (train.day == 6) ]

test.shape
val.shape
train.shape
# 2204180

(519888, 155)

(269737, 155)

(1934443, 155)

In [6]:
#cv
params = {
    'objective': 'binary:logistic',
    'eta': 0.05,
    'colsample_bytree': 0.886,
    'min_child_weight': 1.6,
    'max_depth': 4,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
    'verbose_eval': True,
    'eval_metric': 'logloss',
    'seed': 201803,
    'missing': -1
}

predictors = list(set(predictors + origin))
len(predictors)

xgbtrain = xgb.DMatrix(train[predictors], train[target])

print('Start training')
cv = xgb.cv(
    params,
    dtrain=xgbtrain,
    num_boost_round=5000,
    nfold=3,
    verbose_eval=100,
    early_stopping_rounds=50,
    seed=201803)
print(str(cv.shape[0]) + ':' + str(cv.iloc[-1, :]['test-logloss-mean']))
print('End training')
# 706:0.028848000000000002

132

Start training
[0]	train-logloss:0.645459+6.18241e-06	test-logloss:0.645459+3.74166e-06
[100]	train-logloss:0.0310003+0.000127251	test-logloss:0.0311753+0.000250754
[200]	train-logloss:0.028616+0.000117646	test-logloss:0.0291423+0.000246861
[300]	train-logloss:0.028127+0.00012714	test-logloss:0.028961+0.000242763
[400]	train-logloss:0.0277717+0.000127938	test-logloss:0.028895+0.000245668
[500]	train-logloss:0.0274713+0.000129631	test-logloss:0.0288667+0.000245581
[600]	train-logloss:0.0271903+0.000123489	test-logloss:0.02886+0.000246467
[700]	train-logloss:0.026923+0.000119602	test-logloss:0.0288503+0.00024991
706:0.028848000000000002
End training


In [6]:
#train/val
params = {
    'objective': 'binary:logistic',
    'eta': 0.05,
    'colsample_bytree': 0.886,
    'min_child_weight': 1.6,
    'max_depth': 4,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
    'verbose_eval': True,
    'eval_metric': 'logloss',
    'seed': 201803,
    'missing': -1
}

predictors = list(set(predictors + origin))
len(predictors)

xgbtrain = xgb.DMatrix(train[predictors], train[target])
xgbval = xgb.DMatrix(val[predictors])
# xgbtest = xgb.DMatrix(test[predictors])

print('Start training')
model = xgb.train(params, xgbtrain, num_boost_round=706)

# y_pred = model.predict(xgbtest)
y_pred = model.predict(xgbval)

print(str(log_loss(val.is_trade, y_pred)))
print('End training')
# 0.2922193248704694

132

Start training
0.2922193248704694
End training


In [7]:
val['pred'] = list(y_pred)

val[['instance_id', 'pred', 'is_trade']].to_csv('pred_model3.csv', index=None)

# b榜

In [2]:
result = pd.read_csv('../lgb_v8_0511_ab.txt', sep=' ')

result.shape
result.head()

(1729656, 2)

,instance_id,predicted_score
0,1781317115815731318,0.072283
1,566525167089350640,0.034527
2,147362945230223238,0.086315
3,969157516707841071,0.025213
4,6731403233620820223,0.145889


In [5]:
order = pd.read_csv('round2_ijcai_18_test_b_20180510.txt', sep=' ', usecols=['instance_id'])

order.shape
order.head()

(1209768, 1)

,instance_id
0,74080508196716
1,204576715383250
2,339754209266814
3,1232471723234702
4,1432327895640503


In [8]:
order = order.merge(result, on=['instance_id'], how='left')

order.shape
order.head()

(1209768, 2)

,instance_id,predicted_score
0,74080508196716,0.081938
1,204576715383250,0.026399
2,339754209266814,0.021967
3,1232471723234702,0.097850
4,1432327895640503,0.029386


In [9]:
order.to_csv(
    'resultWithHistory_round(mix3_model1_b_order).txt',
    sep=' ',
    header=['instance_id', 'predicted_score'],
    index=False)